In [14]:
import shutil
import os

model_dir = "saved_movenet_lightning_model_test1"
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)


In [2]:
!pip install tensorflow_hub


  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl (30 kB)
Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)


In [3]:
import tensorflow as tf
import tensorflow_hub as hub

# MoveNet 모델 로드 (저장 없이 직접 사용)
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

def get_keypoints(frame):
    img = cv2.resize(frame, (192, 192))  # Lightning 모델은 192x192 입력
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    input_image = tf.convert_to_tensor(img)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]  # (17, 3)
    return keypoints


In [13]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import requests

# 1. MoveNet Lightning 모델 로드 (서명 방식)
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

def get_keypoints(frame):
    img = cv2.resize(frame, (192, 192))
    img = img.astype(np.int32)  
    img = np.expand_dims(img, axis=0)
    input_image = tf.convert_to_tensor(img, dtype=tf.int32)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0] 
    keypoints_xyc = np.stack([
        keypoints[:, 1],  # x (0~1)
        keypoints[:, 0],  # y (0~1)
        keypoints[:, 2]   # confidence
    ], axis=1).flatten()
    return keypoints_xyc

cap = cv2.VideoCapture(0)
sequence = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("웹캠 프레임을 읽을 수 없습니다.")
        break

    try:
        keypoints = get_keypoints(frame)
        sequence.append(keypoints.tolist())

        # 100프레임마다 추론 서버에 전송
        if len(sequence) == 100:
            data = {"sequence": sequence}
            try:
                res = requests.post("http://localhost:8000/predict", json=data, timeout=3)
                result = res.json()["result"]
                status = ["Normal", "Suspicious", "Fall"][result]
                print(f"예측 결과: {status}") 
                # === Spring Boot로 상태 전송===
                requests.post("http://localhost:8080/fall", json={"status": status})
                # =======================================
            except Exception as e:
                print(f"추론 서버 연결 실패: {e}")
            sequence = []
    except Exception as e:
        print(f"관절 추출 에러: {e}")

    
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


예측 결과: Suspicious
예측 결과: Suspicious


KeyboardInterrupt: 